In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import time
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

path = 'nbdata/data/covid-forecasting-w3/'
train = pd.read_csv(path + 'train.csv')
test  = pd.read_csv(path + 'test.csv')
sub   = pd.read_csv(path + 'submission.csv')

train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])
#path_ext = '../input/novel-corona-virus-2019-dataset/'
#ext_rec = pd.read_csv(path_ext + 'time_series_covid_19_recovered.csv').\
#        melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"], 
#            var_name="Date", 
#            value_name="Recoveries")
#ext_rec['Date'] = ext_rec['Date'].apply(lambda x: (datetime.datetime.strptime(x+"20", '%m/%d/%Y')))
#train = train.merge(ext_rec[['Province/State', 'Country/Region', 'Date', 'Recoveries']], how='left',
#           left_on=['Province/State', 'Country/Region', 'Date'],
#           right_on=['Province/State', 'Country/Region', 'Date'])

# print((train['Date'].dt.date - train['Date'].dt.date.min()))
train['days'] = (train['Date'].dt.date - train['Date'].dt.date.min()).apply(lambda x: x.days)
test['days'] = (test['Date'].dt.date - train['Date'].dt.date.min()).apply(lambda x: x.days)
#train['isTest'] = train['Date'].dt.date >= datetime.date(2020, 3, 12)
#train['isVal'] = np.logical_and(train['Date'].dt.date >= datetime.date(2020, 3, 11), train['Date'].dt.date <= datetime.date(9999, 3, 18))
train.loc[train['Province_State'].isnull(), 'Province_State'] = 'N/A'
test.loc[test['Province_State'].isnull(), 'Province_State'] = 'N/A'

train['Area'] = train['Country_Region'] + '_' + train['Province_State']
test['Area'] = test['Country_Region'] + '_' + test['Province_State']

print(train['Date'].max())
print(test['Date'].min())
print(train['days'].max())
N_AREAS = train['Area'].nunique()
AREAS = np.sort(train['Area'].unique())
#TRAIN_N = 64
TRAIN_N = 77

print(train[train['days'] < TRAIN_N]['Date'].max())
print(train[train['days'] >= TRAIN_N]['Date'].min())
print(train[train['days'] >= TRAIN_N]['Date'].max())
train.head()

test_orig = test.copy()

In [ ]:
train_p_c_raw = train.pivot(index='Area', columns='days', values='ConfirmedCases').sort_index()
train_p_f_raw = train.pivot(index='Area', columns='days', values='Fatalities').sort_index()

train_p_c = np.maximum.accumulate(train_p_c_raw, axis=1)
train_p_f = np.maximum.accumulate(train_p_f_raw, axis=1)

f_rate = (train_p_f / train_p_c).fillna(0)

X_c = np.log(1+train_p_c.values)[:,:TRAIN_N]
X_f = train_p_f.values[:,:TRAIN_N]


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error

def eval1(y, p):
    val_len = y.shape[1] - TRAIN_N
    return np.sqrt(mean_squared_error(y[:, TRAIN_N:TRAIN_N+val_len].flatten(), p[:, TRAIN_N:TRAIN_N+val_len].flatten()))

def run_c(params, X, test_size=50):
    
    gr_base = []
    gr_base_factor = []
    
    x_min = np.ma.MaskedArray(X, X<1)
    x_min = x_min.argmin(axis=1) 
    
    for i in range(X.shape[0]):
        temp = X[i,:]
        threshold = np.log(1+params['min cases for growth rate'])
        num_days = params['last N days']
        if (temp > threshold).sum() > num_days:
            d = np.diff(temp[temp > threshold])[-num_days:]
            w = np.arange(len(d))+1
            w = w**5
            w = w / np.sum(w)
            gr_base.append(np.clip(np.average(d, weights=w), 0, params['growth rate max']))
            d2 = np.diff(d)
            w = np.arange(len(d2))+1
            w = w**10
            w = w / np.sum(w)
            gr_base_factor.append(np.clip(np.average(d2, weights=w), -0.5, params["growth rate factor max"]))
        else:
            gr_base.append(params['growth rate default'])
            gr_base_factor.append(params['growth rate factor'])

    gr_base = np.array(gr_base)
    gr_base_factor = np.array(gr_base_factor)
    #print(gr_base_factor)
    #gr_base = np.clip(gr_base, 0.02, 0.8)
    preds = X.copy()

    for i in range(test_size):
        delta = np.clip(preds[:, -1], np.log(2), None) + gr_base * (1 + params['growth rate factor']*(1 + params['growth rate factor factor'])**(i))**(np.log1p(i))
        #delta = np.clip(preds[:, -1], np.log(2), None) + gr_base * (1 + gr_base_factor*(1 + params['growth rate factor factor'])**(i))**(i)
        #delta = np.clip(preds[:, -1], np.log(2), None) + gr_base * (1 + params['growth rate factor']*(1 + params['growth rate factor factor'])**(i+X.shape[1]-x_min))**(i+X.shape[1]-x_min) 
        preds = np.hstack((preds, delta.reshape(-1,1)))

    return preds

params = {
    "min cases for growth rate": 0,
    "last N days": 15,
    "growth rate default": 0.10,
    "growth rate max": 0.2,
    "growth rate factor max": -0.1,
    "growth rate factor": -0.3,
    "growth rate factor factor": 0.01,
}
#x = train_p_c[train_p_c.index=="Austria_N/A"]

x = train_p_c

preds_c = run_c(params, np.log(1+x.values)[:,:TRAIN_N])
#eval1(np.log(1+x).values, preds_c)

In [ ]:

for i in range(N_AREAS):
    if 'China' in AREAS[i] and preds_c[i, TRAIN_N-1] < np.log(31):
        preds_c[i, TRAIN_N:] = preds_c[i, TRAIN_N-1]


In [ ]:
def lin_w(sz):
    res = np.linspace(0, 1, sz+1, endpoint=False)[1:]
    return np.append(res, np.append([1], res[::-1]))


def run_f(params, X_c, X_f, X_f_r, test_size=50):
  
    X_f_r = np.array(np.ma.mean(np.ma.masked_outside(X_f_r, 0.03, 0.5)[:,:], axis=1))
    X_f_r = np.clip(X_f_r, params['fatality_rate_lower'], params['fatality_rate_upper'])
    #print(X_f_r)
    
    X_c = np.clip(np.exp(X_c)-1, 0, None)
    preds = X_f.copy()
    #print(preds.shape)
    
    train_size = X_f.shape[1] - 1
    for i in range(test_size):
        
        t_lag = train_size+i-params['length']
        t_wsize = 5
        d = np.diff(X_c, axis=1)[:, t_lag-t_wsize:t_lag+1+t_wsize]
#         w = np.arange(d.shape[1])[::-1]+1
#         w = w**1
#         w = w / np.sum(w)
        delta = np.average(d, axis=1)
        #delta = np.average(np.diff(X_c, axis=1)[:, t_lag-t_wsize:t_lag+1+t_wsize], axis=1, weights=lin_w(t_wsize))
        
        delta = params['absolute growth'] + delta * X_f_r
        
        preds = np.hstack((preds, preds[:, -1].reshape(-1,1) + delta.reshape(-1,1)))

    return preds

params = {
    "length": 7,
    "absolute growth": 0.02,
    "fatality_rate_lower": 0.02,
    "fatality_rate_upper": 0.3,
}

preds_f_1 = run_f(params, preds_c, X_f, f_rate.values[:,:TRAIN_N])
preds_f_1 = np.log(1+preds_f_1)

In [ ]:
from torch.utils.data import Dataset
from torch.nn import Parameter
import torch.nn as nn
from torch.nn import init
import math 
import torch
import time

class ZDatasetF(Dataset):
    def __init__(self, X_c, X_f=None, hist_len=10):
        self.X_c = X_c
        self.X_f = X_f
        self.hist_len = hist_len
        self.is_test = X_f is None
    def __len__(self):
        return self.X_c.shape[1]
    def __getitem__(self, idx):
        if self.is_test:
            return {'x_c':self.X_c[:, idx-self.hist_len:idx]}
        else:
            return {'x_c':self.X_c[:, idx-self.hist_len:idx],
                    'x_f':self.X_f[:, idx-1],
                    'y':np.log(1+self.X_f[:, idx])}

class PrLayer2(nn.Module):
    def __init__(self, in_features1, in_features2):
        super(PrLayer2, self).__init__()
        self.weight0 = Parameter(torch.Tensor(1, 1, in_features2))
        self.weight1 = Parameter(torch.Tensor(1, in_features1, in_features2))
        self.reset_parameters()
    def reset_parameters(self):
        init.kaiming_uniform_(self.weight0, a=math.sqrt(5))
        init.kaiming_uniform_(self.weight1, a=math.sqrt(5))
    def forward(self, input):
        return input * torch.sigmoid(self.weight0 + self.weight1)



class ZModelF(nn.Module):

    def __init__(self, hist_len):
        super(ZModelF, self).__init__()
        self.l_conv = PrLayer2(len(X_c),hist_len-1)

    def forward(self, x_c, x_f):
        x = x_c[:,:,1:] - x_c[:,:,:-1]
        res = torch.sum(self.l_conv(x), 2)
        return {'preds': torch.log(1 + x_f + res)}        
        

class DummySampler(torch.utils.data.sampler.Sampler):
    def __init__(self, idx):
        self.idx = idx
    def __iter__(self):
        return iter(self.idx)
    def __len__(self):
        return len(self.idx)
    
    
def _smooth_l1_loss(target):
    t = torch.abs(target)
    t = torch.where(t < 1, 0.5 * t ** 2, t - 0.5)
    return torch.mean(t)


n_epochs = 5000
lr = 0.18
bag_size = 4
device = 'cpu'
hist_len = 14
loss_func = torch.nn.MSELoss()
reg_loss_func = _smooth_l1_loss
reg_factor = 0.035


train_dataset = ZDatasetF(np.exp(X_c)-1, X_f, hist_len=hist_len)
test_dataset = ZDatasetF(np.exp(preds_c)-1, hist_len=hist_len)

#trn_idx = np.arange(hist_len+1, len(train_dataset))
trn_idx = np.arange(hist_len+1, len(train_dataset))
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(trn_idx)
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, sampler=train_sampler, num_workers=0, pin_memory=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(trn_idx), sampler=train_sampler, num_workers=0, pin_memory=True)

test_idx = np.arange(TRAIN_N, len(test_dataset))
test_sampler = DummySampler(test_idx)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, sampler=test_sampler, num_workers=0, pin_memory=True)


#gradient_accumulation = len(trn_idx)
gradient_accumulation = 1

preds_f = 0

for m_i in range(bag_size):
    model_f = ZModelF(hist_len=hist_len).to(device)
    optimizer_f = torch.optim.Adam(model_f.parameters(), lr=lr)
    model_f.train()

    start_time = time.time()
    for epoch in range(n_epochs):

        s = time.time()
        avg_train_loss = 0
        
        optimizer_f.zero_grad()
        for idx, data in enumerate(train_loader):

            X1 = data['x_c'].to(device).float()
            X2 = data['x_f'].to(device).float()
            y = data['y'].to(device).float()
            
            preds = model_f(X1, X2)['preds'].float()

            cond = X2 > np.log(10)
            preds = preds[cond]
            y = y[cond]
            
            loss = loss_func(preds, y)
            
            loss += reg_factor * reg_loss_func(model_f.l_conv.weight1)
            
            avg_train_loss += loss  / len(train_loader)
            
            loss.backward()
            if (idx+1) % gradient_accumulation == 0 or idx == len(train_loader) - 1: 
                optimizer_f.step()
                optimizer_f.zero_grad()
                
        #if epoch % 1000 == 0:
        if False:
        
            model_f.eval()
            preds_f_delta = train_p_f.values[:,:TRAIN_N]

            for idx, data in enumerate(test_loader):
                X1 = data['x_c'].to(device).float()
                temp = model_f(X1, torch.Tensor(preds_f_delta[:,-1]).unsqueeze(0))['preds']
                temp = np.exp(temp.detach().cpu().numpy().reshape(-1,1)) - 1
                preds_f_delta = np.hstack((preds_f_delta, temp))

            preds_f_delta = np.log(1 + preds_f_delta)
            val_len = train_p_c.values.shape[1] - TRAIN_N

            m2 = np.sqrt(mean_squared_error(np.log(1 + train_p_f_raw.values[:, TRAIN_N:TRAIN_N+val_len]).flatten(), \
                                            preds_f_delta[:, TRAIN_N:TRAIN_N+val_len].flatten()))
            print(f"{epoch:2} train_loss {avg_train_loss:<8.4f} val_loss {m2:8.5f} {time.time()-s:<2.2f}")
                
            model_f.train()
        
    model_f.eval()
    preds_f_delta = train_p_f.values[:,:TRAIN_N]
    
    for idx, data in enumerate(test_loader):
        X1 = data['x_c'].to(device).float()
        temp = model_f(X1, torch.Tensor(preds_f_delta[:,-1]).unsqueeze(0))['preds']
        temp = np.exp(temp.detach().cpu().numpy().reshape(-1,1)) - 1
        preds_f_delta = np.hstack((preds_f_delta, temp))
    preds_f += preds_f_delta / bag_size

preds_f_2 = np.log(1 + preds_f)

print("Done")

#eval1(np.log(1+train_p_f).values, preds_f_2)


In [ ]:
preds_f = np.average([preds_f_1, preds_f_2], axis=0, weights=[1,2])

In [ ]:
from sklearn.metrics import mean_squared_error

if False:
    val_len = train_p_c.values.shape[1] - TRAIN_N

    for i in range(val_len):
        d = i + TRAIN_N
        m1 = np.sqrt(mean_squared_error(np.log(1 + train_p_c_raw.values[:, d]), preds_c[:, d]))
        m2 = np.sqrt(mean_squared_error(np.log(1 + train_p_f_raw.values[:, d]), preds_f[:, d]))
        print(f"{d}: {(m1 + m2)/2:8.5f} [{m1:8.5f} {m2:8.5f}]")

    print()

    m1 = np.sqrt(mean_squared_error(np.log(1 + train_p_c_raw.values[:, TRAIN_N:TRAIN_N+val_len]).flatten(), preds_c[:, TRAIN_N:TRAIN_N+val_len].flatten()))
    m2 = np.sqrt(mean_squared_error(np.log(1 + train_p_f_raw.values[:, TRAIN_N:TRAIN_N+val_len]).flatten(), preds_f[:, TRAIN_N:TRAIN_N+val_len].flatten()))
    print(f"{(m1 + m2)/2:8.5f} [{m1:8.5f} {m2:8.5f}]")

In [ ]:
import matplotlib.pyplot as plt

plt.style.use(['default'])
fig = plt.figure(figsize = (15, 5))

#idx = worst_idx
#print(AREAS[idx])

idx = np.where(AREAS == 'Austria_N/A')[0][0]
plt.plot(np.log(1+train_p_c.values[idx]), label=AREAS[idx], color='darkblue')
plt.plot(preds_c[idx], linestyle='--', color='darkblue')

idx = np.where(AREAS == 'Germany_N/A')[0][0]
plt.plot(np.log(1+train_p_c.values[idx]), label=AREAS[idx], color='red')
plt.plot(preds_c[idx], linestyle='--', color='red')


idx = np.where(AREAS == 'China_Hubei')[0][0]
plt.plot(np.log(1+train_p_c.values[idx]), label=AREAS[idx], color='grey')
plt.plot(preds_c[idx], linestyle='--', color='grey')


idx = np.where(AREAS == 'Iran_N/A')[0][0]
plt.plot(np.log(1+train_p_c.values[idx]), label=AREAS[idx], color='green')
plt.plot(preds_c[idx], linestyle='--', color='green')


idx = np.where(AREAS == 'Japan_N/A')[0][0]
plt.plot(np.log(1+train_p_c.values[idx]), label=AREAS[idx], color='purple')
plt.plot(preds_c[idx], linestyle='--', color='purple')


idx = np.where(AREAS == 'Brazil_N/A')[0][0]
plt.plot(np.log(1+train_p_c.values[idx]), label=AREAS[idx], color='black')
plt.plot(preds_c[idx], linestyle='--', color='black')


idx = np.where(AREAS == 'Denmark_N/A')[0][0]
plt.plot(np.log(1+train_p_c.values[idx]), label=AREAS[idx], color='yellow')
plt.plot(preds_c[idx], linestyle='--', color='yellow')

idx = np.where(AREAS == 'Italy_N/A')[0][0]
plt.plot(np.log(1+train_p_c.values[idx]), label=AREAS[idx], color='blue')
plt.plot(preds_c[idx], linestyle='--', color='blue')


plt.legend()
# plt.show()
plt.clf()

In [ ]:
import matplotlib.pyplot as plt

plt.style.use(['default'])
fig = plt.figure(figsize = (15, 5))

#idx = worst_idx
#print(AREAS[idx])

idx = np.where(AREAS == 'Austria_N/A')[0][0]
plt.plot(np.log(1+train_p_f.values[idx]), label=AREAS[idx], color='darkblue')
plt.plot(preds_f[idx], linestyle='--', color='darkblue')

idx = np.where(AREAS == 'Germany_N/A')[0][0]
plt.plot(np.log(1+train_p_f.values[idx]), label=AREAS[idx], color='red')
plt.plot(preds_f[idx], linestyle='--', color='red')


idx = np.where(AREAS == 'China_Hubei')[0][0]
plt.plot(np.log(1+train_p_f.values[idx]), label=AREAS[idx], color='grey')
plt.plot(preds_f[idx], linestyle='--', color='grey')


idx = np.where(AREAS == 'Iran_N/A')[0][0]
plt.plot(np.log(1+train_p_f.values[idx]), label=AREAS[idx], color='green')
plt.plot(preds_f[idx], linestyle='--', color='green')


idx = np.where(AREAS == 'Japan_N/A')[0][0]
plt.plot(np.log(1+train_p_f.values[idx]), label=AREAS[idx], color='purple')
plt.plot(preds_f[idx], linestyle='--', color='purple')


idx = np.where(AREAS == 'Brazil_N/A')[0][0]
plt.plot(np.log(1+train_p_f.values[idx]), label=AREAS[idx], color='black')
plt.plot(preds_f[idx], linestyle='--', color='black')


idx = np.where(AREAS == 'Denmark_N/A')[0][0]
plt.plot(np.log(1+train_p_f.values[idx]), label=AREAS[idx], color='yellow')
plt.plot(preds_f[idx], linestyle='--', color='yellow')

idx = np.where(AREAS == 'Italy_N/A')[0][0]
plt.plot(np.log(1+train_p_f.values[idx]), label=AREAS[idx], color='blue')
plt.plot(preds_f[idx], linestyle='--', color='blue')

plt.legend()
plt.clf()
# plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.style.use(['default'])
fig = plt.figure(figsize = (15, 5))

idx = np.random.choice(N_AREAS)
print(AREAS[idx])

plt.plot(np.log(1+train_p_c.values[idx]), label=AREAS[idx], color='darkblue')
plt.plot(preds_c[idx], linestyle='--', color='darkblue')

# plt.show()
plt.clf()

In [ ]:
EU_COUNTRIES = ['Austria', 'Italy', 'Belgium', 'Latvia', 'Bulgaria', 'Lithuania', 'Croatia', 'Luxembourg', 'Cyprus', 'Malta', 'Czechia', 
                'Netherlands', 'Denmark', 'Poland', 'Estonia', 'Portugal', 'Finland', 'Romania', 'France', 'Slovakia', 'Germany', 'Slovenia', 
                'Greece', 'Spain', 'Hungary', 'Sweden', 'Ireland']
EUROPE_OTHER = ['Albania', 'Andorra', 'Bosnia and Herzegovina', 'Liechtenstein', 'Monaco', 'Montenegro', 'North Macedonia',
                'Norway', 'San Marino', 'Serbia', 'Switzerland', 'Turkey', 'United Kingdom']
AFRICA = ['Algeria', 'Burkina Faso', 'Cameroon', 'Congo (Kinshasa)', "Cote d'Ivoire", 'Egypt', 'Ghana', 'Kenya', 'Madagascar',
                'Morocco', 'Nigeria', 'Rwanda', 'Senegal', 'South Africa', 'Togo', 'Tunisia', 'Uganda', 'Zambia']
NORTH_AMERICA = ['US', 'Canada', 'Mexico']
SOUTH_AMERICA = ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Paraguay', 'Peru', 'Uruguay', 'Venezuela']
MIDDLE_EAST = ['Afghanistan', 'Bahrain', 'Iran', 'Iraq', 'Israel', 'Jordan', 'Kuwait', 'Lebanon', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates']
ASIA = ['Bangladesh', 'Brunei', 'Cambodia', 'India', 'Indonesia', 'Japan', 'Kazakhstan', 'Korea, South', 'Kyrgyzstan', 'Malaysia',
                'Pakistan', 'Singapore', 'Sri Lanka', 'Taiwan*', 'Thailand', 'Uzbekistan', 'Vietnam']

In [ ]:
import matplotlib.pyplot as plt

def plt1(ar, ar2, ax, col='darkblue', linew=0.2):
    ax.plot(ar2, linestyle='--', linewidth=linew/2, color=col)
    ax.plot(np.log(1+ar), linewidth=linew, color=col)

plt.style.use(['default'])
fig, axs = plt.subplots(3, 2, figsize=(18, 15), sharey=True)

X = train_p_c.values
#X = train_p_f.values

for ar in range(X.shape[0]):
    
    temp = X[ar]
    temp2 = preds_c[ar]
    if 'China' in AREAS[ar]:
        plt1(temp, temp2, axs[0,0])
    elif AREAS[ar].split('_')[0] in NORTH_AMERICA:
        plt1(temp, temp2, axs[0,1])
    elif AREAS[ar].split('_')[0] in EU_COUNTRIES + EUROPE_OTHER:
        plt1(temp, temp2, axs[1,0])
    elif AREAS[ar].split('_')[0] in SOUTH_AMERICA + AFRICA:
        plt1(temp, temp2, axs[1,1])
    elif AREAS[ar].split('_')[0] in MIDDLE_EAST + ASIA:
        plt1(temp, temp2, axs[2,0])
    else:
        plt1(temp, temp2, axs[2,1])

print("Confirmed Cases")
axs[0,0].set_title('China')
axs[0,1].set_title('North America')
axs[1,0].set_title('Europe')
axs[1,1].set_title('Africa + South America')
axs[2,0].set_title('Asia + Middle East')
axs[2,1].set_title('Other')
plt.clf()
# plt.show()

In [ ]:
import matplotlib.pyplot as plt

def plt1(ar, ar2, ax, col='darkblue', linew=0.2):
    ax.plot(ar2, linestyle='--', linewidth=linew/2, color=col)
    ax.plot(np.log(1+ar), linewidth=linew, color=col)

plt.style.use(['default'])
fig, axs = plt.subplots(3, 2, figsize=(18, 15), sharey=True)

#X = train_p_c.values
X = train_p_f.values

for ar in range(X.shape[0]):
    
    temp = X[ar]
    temp2 = preds_f[ar]
    if 'China' in AREAS[ar]:
        plt1(temp, temp2, axs[0,0])
    elif AREAS[ar].split('_')[0] in NORTH_AMERICA:
        plt1(temp, temp2, axs[0,1])
    elif AREAS[ar].split('_')[0] in EU_COUNTRIES + EUROPE_OTHER:
        plt1(temp, temp2, axs[1,0])
    elif AREAS[ar].split('_')[0] in SOUTH_AMERICA + AFRICA:
        plt1(temp, temp2, axs[1,1])
    elif AREAS[ar].split('_')[0] in MIDDLE_EAST + ASIA:
        plt1(temp, temp2, axs[2,0])
    else:
        plt1(temp, temp2, axs[2,1])

print("Fatalities")
axs[0,0].set_title('China')
axs[0,1].set_title('North America')
axs[1,0].set_title('Europe')
axs[1,1].set_title('Africa + South America')
axs[2,0].set_title('Asia + Middle East')
axs[2,1].set_title('Other')
# plt.show()
plt.clf()

In [ ]:

temp = pd.DataFrame(np.clip(np.exp(preds_c) - 1, 0, None))
temp['Area'] = AREAS
temp = temp.melt(id_vars='Area', var_name='days', value_name="ConfirmedCases")

test = test_orig.merge(temp, how='left', left_on=['Area', 'days'], right_on=['Area', 'days'])

temp = pd.DataFrame(np.clip(np.exp(preds_f) - 1, 0, None))
temp['Area'] = AREAS
temp = temp.melt(id_vars='Area', var_name='days', value_name="Fatalities")

test = test.merge(temp, how='left', left_on=['Area', 'days'], right_on=['Area', 'days'])
test.head()

In [ ]:
# test.to_csv("submission.csv", index=False, columns=["ForecastId", "ConfirmedCases", "Fatalities"])

In [ ]:
test.days.nunique()

In [ ]:
for i, rec in test.groupby('Area').last().sort_values("ConfirmedCases", ascending=False).iterrows():
    print(f"{rec['ConfirmedCases']:10.1f} {rec['Fatalities']:10.1f}  {rec['Country_Region']}, {rec['Province_State']}")


In [ ]:
print(f"{test.groupby('Area')['ConfirmedCases'].last().sum():10.1f}")
print(f"{test.groupby('Area')['Fatalities'].last().sum():10.1f}")

In [ ]:
test_p_c = test.pivot(index='Area', columns='days', values='ConfirmedCases').sort_index().values
test_p_f = test.pivot(index='Area', columns='days', values='Fatalities').sort_index().values
dates = test.Date.dt.strftime('%d.%m.%Y').unique()

In [ ]:
print("Confirmed Cases")
for i in [7,14,21,28,35,42]:
    print(f'week{i//7-1}  ', dates[i],  f'   {round(test_p_c[:,i].sum(),0):,}')

In [ ]:
print("Fatalities")
for i in [7,14,21,28,35,42]:
    print(f'week{i//7-1}  ', dates[i],  f'   {round(test_p_f[:,i].sum(),0):,}', )